In [1]:
import cutiepy
import qutip
import numpy as np
import scipy
N = 100
ts = np.linspace(0,1000,100)

#qutip

In [2]:
init = qutip.coherent(N,2)
H = qutip.num(N)
opt = qutip.Options(rhs_reuse=True)
%time res = qutip.sesolve(H,init,ts,[], options=opt)
%time res = qutip.sesolve(H,init,ts,[], options=opt)
print(res.states[-1].data.A[:4], res.states[-1].norm())

CPU times: user 2.04 s, sys: 240 ms, total: 2.28 s
Wall time: 2 s
CPU times: user 1.96 s, sys: 4 ms, total: 1.97 s
Wall time: 1.96 s
[[ 0.13538037+0.j        ]
 [ 0.15226989-0.22388607j]
 [-0.14070393-0.35612185j]
 [-0.43138871-0.09691328j]] 1.0


#cutiepy

In [3]:
init = cutiepy.coherent(N,2)
H = cutiepy.num(N)
%time res = cutiepy.sesolve(H, init, np.linspace(0,1000,100))
%time res = cutiepy.sesolve(H, init, np.linspace(0,1000,100))
print(res[-1].numerical[:4], np.linalg.norm(res[-1].numerical))

CPU times: user 5.44 s, sys: 40 ms, total: 5.48 s
Wall time: 9.32 s
CPU times: user 4.05 s, sys: 12 ms, total: 4.06 s
Wall time: 4.06 s
[[ 0.13533528+0.j        ]
 [ 0.20064000+0.18167593j]
 [ 0.03788156+0.38090694j]
 [-0.26279469+0.35539523j]] 0.999999596058


#scipy (dense)

In [4]:
%%time
H = qutip.num(N).data.A
op = -1j*H
f = lambda t, state: op.dot(state)
ode = scipy.integrate.ode(f)                                             
ode.set_integrator('zvode', method='adams', rtol=1e-6, atol=1e-8, )                                      
state0_num = qutip.coherent(N,2).data.A
ode.set_initial_value(state0_num, ts[0])                                     
res = [state0_num]                                                              
for t in ts[1:]:                                                             
    ode.integrate(t)                                                            
    if not ode.successful():                                                    
        RuntimeError('The ODE solver failed.') 
    #y = ode.y                    
    #n = scipy.linalg.norm(y)     
    #y /= n                       
    #ode.set_initial_value(y, t)  
    res.append(ode.y)                                                           
print(res[-1][:4], np.linalg.norm(res[-1]))

[[ 0.13533528+0.j        ]
 [ 0.07067765-0.26127943j]
 [-0.33058338-0.19296997j]
 [-0.31476474+0.31029364j]] 0.999782165209
CPU times: user 3.56 s, sys: 3.88 s, total: 7.45 s
Wall time: 1.95 s


/home/stefan/python_lab/lib/python3.4/site-packages/scipy/integrate/_ode.py:853: UserWarning: zvode: Excess work done on this call. (Perhaps wrong MF.)
  'Unexpected istate=%s' % istate))
